In [1]:

import pandas as pd
from glob import glob
from tqdm import notebook

In [2]:
import pymysql as m
import time
import traceback

In [3]:
def ErrorLog_sql_dtm(error: str, element = None):
    current_time = time.strftime("%Y.%m.%d/%H:%M:%S", time.localtime(time.time()))
    with open("error_Log_sql_dtm.txt", "a") as f:
        f.write(f"[{current_time}] - {error}\n")
        if element != None:
            f.write(f"[{element}\n\n")

In [4]:
def send_mysql_select_for_fit():
    con = m.connect(host = "localhost", user = 'root', db = 'mintsaveworld' , password = "", charset = "utf8mb4")
    cur = con.cursor()
    
    sql = """select link, token from society;"""
        
    cur.execute(sql)
    result = cur.fetchall()
    
    con.commit()
    con.close()

    return result

In [5]:
### table name = society_dtm
def send_mysql_insert_tdm(dtms, linklist):
    
    
    con = m.connect(host = "localhost", user = 'root', db = 'mintsaveworld' , password = "", charset = "utf8mb4")
    cur = con.cursor()
    idx = 0
    for dtm in notebook.tqdm(dtms):
        
        q_link = linklist[idx]
        q_data = str(list(dtm.data))
        q_indices = str(list(dtm.indices))
        q_indptr = str(list(dtm.indptr))



        sql = """insert into society_dtm values (%s,%s,%s,%s); """
        
        try :

            cur.execute(sql,(q_link, q_data, q_indices, q_indptr))
        except:
            err = traceback.format_exc().split('\n')[-2]
            element = q_link + q_data + q_indices + q_indptr
            ErrorLog_sql_dtm(err, )
            
        idx = idx + 1
            
    con.commit()
    con.close()

In [10]:
def send_mysql_insert_tdm_shape(shape):
    con = m.connect(host = "localhost", user = 'root', db = 'mintsaveworld' , password = "", charset = "utf8mb4")
    cur = con.cursor()

    q_shape = shape
    
    sql = """insert into society_dtm_set values (%s); """
    
    try :
        cur.execute(sql,(q_shape))
    except:
            err = traceback.format_exc().split('\n')[-2]
            ErrorLog_sql_dtm(err)
            
    con.commit()
    con.close()
    
    
    

In [7]:
result = send_mysql_select_for_fit()

In [8]:
len(result)

1406520

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [10]:
tfidf_vec = TfidfVectorizer()

In [11]:
%%time
tfidf_vec.fit(list(zip(*result))[1])

Wall time: 2min 58s


TfidfVectorizer()

In [12]:
%%time
tfidf_dtm = tfidf_vec.transform(list(zip(*result))[1])

Wall time: 3min 3s


In [13]:
with open("../model/pickled_tfidf_vec.bin", "wb") as f:
    pickle.dump(tfidf_vec, f)

In [14]:
with open("../model/pickled_tfidf_dtm.bin", "wb") as f:
    pickle.dump(tfidf_dtm, f)

In [8]:
# 필요 시 사용
import pickle
with open("../model/pickled_tfidf_dtm.bin", "rb") as f:
    tfidf_dtm = pickle.load(f)

In [9]:
send_mysql_insert_tdm(tfidf_dtm, list(zip(*result))[0])

0it [00:00, ?it/s]

In [11]:
send_mysql_insert_tdm_shape(tfidf_dtm.shape[1])

In [ ]:
#linklist list(zip(*result))[0]